In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
def sep():
    print("\n" + "-"*50 + "\n")

In [106]:
A = np.random.randint(0, 5, size=(6, 6))
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in A]))

4	0	4	1	1	0
3	1	1	1	0	0
4	0	0	3	2	1
1	3	4	2	4	0
4	3	0	3	0	1
2	3	4	4	2	2


In [107]:
kernel = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
print('\n'.join(['\t'.join([str(cell) for cell in row]) for row in kernel]))

1	1	1
1	1	1
1	1	1


In [108]:
def conv2d(A, kernel):
    A_row, A_col = A.shape
    kernel_row, kernel_col = kernel.shape
    result = np.zeros((A_row - kernel_row + 1, A_col - kernel_col + 1))
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            result[i][j] = np.sum(A[i:i+kernel_row, j:j+kernel_col] * kernel)
    return result

In [109]:
def pad(matrix, padding_size):
    original_rows, original_cols = matrix.shape

    # Create a new matrix filled with zeros, large enough to include the padding
    padded_matrix = np.zeros((original_rows + 2 * padding_size, original_cols + 2 * padding_size))
    
    # Place the original matrix into the center of the padded matrix
    padded_matrix[padding_size:-padding_size, padding_size:-padding_size] = matrix
    
    # Return the padded matrix
    return padded_matrix


In [110]:
A = pad(A, 1)
print(A)
print()
print(conv2d(A, kernel))

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 4. 0. 4. 1. 1. 0. 0.]
 [0. 3. 1. 1. 1. 0. 0. 0.]
 [0. 4. 0. 0. 3. 2. 1. 0.]
 [0. 1. 3. 4. 2. 4. 0. 0.]
 [0. 4. 3. 0. 3. 0. 1. 0.]
 [0. 2. 3. 4. 4. 2. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]

[[ 8. 13.  8.  8.  3.  1.]
 [12. 17. 11. 13.  9.  4.]
 [12. 17. 15. 17. 13.  7.]
 [15. 19. 18. 18. 16.  8.]
 [16. 24. 26. 23. 18.  9.]
 [12. 16. 17. 13. 12.  5.]]


In [111]:
class ConvolutionLayer:
    def __init__(self, kernel_size, num_filters):
        self.kernel_size = kernel_size
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, kernel_size, kernel_size) / (kernel_size * kernel_size)

    def generate_patches(self, A):
        A_row, A_col = A.shape
        for i in range(A_row - self.kernel_size + 1):
            for j in range(A_col - self.kernel_size + 1):
                patch = A[i:i+self.kernel_size, j:j+self.kernel_size]
                yield patch, i, j
    
    def forward(self, A):
        A_row, A_col = A.shape
        output = np.zeros((A_row - self.kernel_size + 1, A_col - self.kernel_size + 1, self.num_filters))

        for patch, i, j in self.generate_patches(A):
            output[i, j] = np.sum(patch * self.filters, axis=(1, 2))
        
        return output

In [113]:
cnn_layer = ConvolutionLayer(3, 2)
print(cnn_layer.filters.shape)
print(cnn_layer.filters)
sep()

patches = cnn_layer.generate_patches(A)
print(next(patches))
sep()

output = cnn_layer.forward(A)
print(output.shape)
print(output[:, :, 0])
sep()

(2, 3, 3)
[[[-0.06584703  0.10801163  0.07549601]
  [ 0.15854777 -0.07494226 -0.01909331]
  [ 0.05528397 -0.08124301 -0.0266434 ]]

 [[-0.07270484  0.00721706 -0.08292324]
  [ 0.14406211 -0.07407455  0.01159387]
  [ 0.02016602  0.10886488 -0.04309509]]]

--------------------------------------------------

(array([[0., 0., 0.],
       [0., 4., 0.],
       [0., 3., 1.]]), 0, 0)

--------------------------------------------------

(6, 6, 2)
[[-0.57014149  0.61578336 -0.3714648   0.51419648  0.13888948  0.15854777]
 [-0.13684562  0.64133955  0.49212454 -0.2932908   0.17743485 -0.03652211]
 [-0.06141136  0.32513899 -0.15202626 -0.26877255  0.02641435  0.46328916]
 [-0.10507791 -0.42863351  0.45009739  0.63923121  0.2505132   0.52926566]
 [-0.26496554  0.7298015   0.53816711 -0.12733059  0.76226554 -0.39024846]
 [ 0.4513701   0.07654233  0.12844799  0.62027034  0.32407489  0.27522266]]

--------------------------------------------------



In [114]:
class MaxPool2d:
    def __init__(self, pool_size):
        self.pool_size = pool_size
    
    def generate_patches(self, A):
        A_row, A_col, num_filters = A.shape
        for i in range(0, A_row, self.pool_size):
            for j in range(0, A_col, self.pool_size):
                patch = A[i : i+  self.pool_size, j : j + self.pool_size]
                yield patch, i, j
    
    def forward(self, A):
        A_row, A_col, num_filters = A.shape
        output = np.zeros((A_row // self.pool_size, A_col // self.pool_size, num_filters))

        for patch, i, j in self.generate_patches(A):
            output[i//self.pool_size, j//self.pool_size] = np.amax(patch, axis=(0, 1))
        
        return output

In [115]:
max_pool = MaxPool2d(2)
output = max_pool.forward(output)
print(output.shape)
print(output[:, :, 0])

(3, 3, 2)
[[0.64133955 0.51419648 0.17743485]
 [0.32513899 0.63923121 0.52926566]
 [0.7298015  0.62027034 0.76226554]]


In [119]:
class CNN:
    def __init__(self, num_filters, kernel_size, pool_size):
        self.conv = ConvolutionLayer(kernel_size, num_filters)
        self.pool = MaxPool2d(pool_size)
    
    def forward(self, A):
        output = self.conv.forward(A)
        output = self.pool.forward(output)
        return output

In [120]:
mnist = pd.read_csv('mnist_train.csv')
mnist.head()